In [1]:
import pandas as pd
import vnstock
from vnstock import *
from datetime import datetime
import multiprocessing as mp
import math
from queue import Queue
from threading import Thread
concurrent = 200

In [8]:
stock_symbols = pd.read_csv("company.csv")['ticker'].tolist()
q=Queue()
for ele in stock_symbols:
    q.put(ele)
def divide_list(input_list, num_sublists):
    sublist_length = math.ceil(len(input_list) / num_sublists)
    return [input_list[i:i+sublist_length] for i in range(0, len(input_list), sublist_length)]

In [47]:
# today = str(datetime.now().date() - timedelta(days=1))
today = '2024-05-03'
def get_data(stock_symbols):
    print(len(stock_symbols))
    df=pd.DataFrame()
    for stock_symbol in (stock_symbols):
        

        try:
            real_time_data = stock_historical_data(symbol=stock_symbol, start_date= today , end_date= today , resolution="15", type="stock", beautify=False, decor=False, source='DNSE')
            if not real_time_data.empty:
                a = real_time_data.iloc[-1]
                df = df._append(a, ignore_index=True)
                # print(a)
                # print(len(df))
                # print(stock_symbol)
            else:
                print(stock_symbol +' empty')
        except Exception as e:
            print(f"Error processing stock symbol {stock_symbol}: {str(e)}")
            continue
    # print(df)
    return df
with mp.Pool() as pool:
    df1=pool.starmap(get_data,zip(divide_list(stock_symbols,2)))


 

805806



VVS empty
MTC empty
VMK empty
MTG empty
THM empty
BVL empty
NAC empty
FHN empty
FUCTVGF3 empty
NAP empty
NTF empty
CNA empty
NAW empty
DSD empty
MGR empty
ND2 empty
FUCTVGF4 empty
MND empty
FUEKIVFS empty
NDF empty
FUEBFVND empty
NDW empty
FUCTVGF5 empty
NDX empty
D17 empty
NFC empty
NGC empty
MNB empty
VCE empty
ACS empty
NLS empty
NNT empty
ALT empty
NOS empty
AMD empty
NQB empty
NQT empty
AMP empty
NST empty
APC empty
NTB empty
APL empty
NWT empty
ARM empty
ASA empty
ONW empty
AVF empty
PAI empty
B82 empty
PAP empty
BCV empty
PCC empty
PCG empty
PCM empty
PEQ empty
BBH empty
BBS empty
BBT empty
BCB empty
PEN empty
BDB empty
BED empty
PGS empty
BEL empty
NBT empty
PHS empty
BGW empty
PIA empty
BHC empty
PIC empty
BHK empty
PID empty
BHP empty
PIS empty
BHG empty
PJC empty
BIO empty
PJS empty
PJT empty
DTB empty
PLE empty
BMD empty
PMJ empty
BMG empty
PMP empty
BMJ empty
PMT empty
BMN empty
PNG empty
MBN empty
PND empty
BNW empty
PNT empty
BRS empty
POB empty
BSP empty
POV empty
BT6 e

In [67]:
df=pd.concat(df1,ignore_index=True)
df

,time,open,high,low,close,volume,ticker
0,2024-05-03 14:45:00,29.00,29.90,29.00,29.90,33600,HIO
1,2024-05-03 14:45:00,16.15,16.15,16.15,16.15,2600,SBG
2,2024-05-03 14:45:00,4.40,4.40,4.40,4.40,100,HSV
3,2024-05-03 14:45:00,22.00,22.00,21.80,22.00,9800,CST
4,2024-05-03 14:15:00,13.40,13.40,13.40,13.40,200,SGI
...,...,...,...,...,...,...,...
1043,2024-05-03 10:45:00,20.00,20.00,20.00,20.00,100,DFC
1044,2024-05-03 14:30:00,7.20,7.20,7.20,7.20,100,XMC
1045,2024-05-03 09:15:00,7.40,7.40,7.40,7.40,1000,XPH
1046,2024-05-03 09:30:00,23.50,23.50,23.50,23.50,100,XHC


In [68]:
import numpy as np
df['time']=pd.to_datetime(df['time']).dt.tz_localize('Asia/Ho_Chi_Minh').values.astype(np.int64) // 10 ** 9




,time,open,high,low,close,volume,ticker
0,1714722300,29.00,29.90,29.00,29.90,33600,HIO
1,1714722300,16.15,16.15,16.15,16.15,2600,SBG
2,1714722300,4.40,4.40,4.40,4.40,100,HSV
3,1714722300,22.00,22.00,21.80,22.00,9800,CST
4,1714720500,13.40,13.40,13.40,13.40,200,SGI
...,...,...,...,...,...,...,...
1043,1714707900,20.00,20.00,20.00,20.00,100,DFC
1044,1714721400,7.20,7.20,7.20,7.20,100,XMC
1045,1714702500,7.40,7.40,7.40,7.40,1000,XPH
1046,1714703400,23.50,23.50,23.50,23.50,100,XHC


In [69]:
import socket
from confluent_kafka import Producer
conf = {'bootstrap.servers': 'pkc-ldvr1.asia-southeast1.gcp.confluent.cloud:9092',
        'security.protocol': 'SASL_SSL',
        'sasl.mechanism': 'PLAIN',
        'sasl.username': 'MPT6ORJOO7EOIBBT',
        'sasl.password': '/NjGhfqfDTWlLmh2d5iUeKdqz6Bm9POPBEH8S9fv0BiRsIXGmn68CYqPI1lfiOUA',
        'client.id': socket.gethostname()}
producer = Producer(conf)

In [70]:
def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

def send_to_kafka(producer, topic, key, message):
    # Sending a message to Kafka
    # producer.produce(topic, key=key, partition=0, value=json.dumps(message).encode("utf-8"))
    # print(producer, topic, key, message)
    producer.produce(topic, key=key, value=json.dumps(message).encode("utf-8"), callback=delivery_report)
    producer.flush()


In [74]:
for index, row in df.iterrows():
    row=row.to_dict()
    print(index)
    send_to_kafka(producer=producer,topic='topic_0',key=row['ticker'],message=row)


0
Message delivered to topic_0 [1]
1
Message delivered to topic_0 [5]
2
Message delivered to topic_0 [4]
3
Message delivered to topic_0 [5]
4
Message delivered to topic_0 [3]
5
Message delivered to topic_0 [2]
6
Message delivered to topic_0 [2]
7
Message delivered to topic_0 [0]
8
Message delivered to topic_0 [4]
9
Message delivered to topic_0 [2]
10
Message delivered to topic_0 [5]
11
Message delivered to topic_0 [3]
12
Message delivered to topic_0 [0]
13
Message delivered to topic_0 [1]
14
Message delivered to topic_0 [1]
15
Message delivered to topic_0 [0]
16
Message delivered to topic_0 [4]
17
Message delivered to topic_0 [4]
18
Message delivered to topic_0 [1]
19
Message delivered to topic_0 [5]
20
Message delivered to topic_0 [1]
21
Message delivered to topic_0 [4]
22
Message delivered to topic_0 [2]
23
Message delivered to topic_0 [4]
24
Message delivered to topic_0 [1]
25
Message delivered to topic_0 [5]
26
Message delivered to topic_0 [4]
27
Message delivered to topic_0 [5]
28